In [56]:
import embed_anything
import os
import time
from pinecone import Pinecone
import numpy as np

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "YOUR KEY HERE"

In [57]:
data = embed_anything.embed_directory("test_files", embeder="OpenAI")
embeddings = np.array([data.embedding for data in data])


In [59]:
pc = Pinecone(api_key="YOUR_API_KEY")
index = pc.Index("anything")

In [44]:
for i in range(len(data)):
    index.upsert(
        vectors=[{"id": str(i), "values": data[i].embedding, "metadata": {"text": data[i].text}}]
    )

In [60]:
query = "What is AI?"
query_embedding = embed_anything.embed_query([query], embeder="OpenAI")

In [62]:
def retrieval(query):
    query_embedding = embed_anything.embed_query([query], embeder="OpenAI")
    return index.query(queries=[query_embedding[0].embedding], top_k=5)
    

In [175]:
def retrieval(query):
    query_embedding = embed_anything.embed_query(query, embeder="OpenAI")
    context =  index.query(vector=query_embedding[0].embedding, top_k=2)
    indices = [int(context.matches[i]['id']) for i in range(len(context.matches))]
    return indices
    


# for i in range(len(matches)):
#     print(index.fetch[i].vectors[i].metadata['text'])


In [176]:
indices = retrieval(["what is AI?"])

In [177]:
def get_text(indices):
    return [index.fetch([str(e)])['vectors'][str(e)]['metadata']['text'] for e in indices]



In [178]:
content = query + " "
for i in range(min(len(indices), 3)):
    content += get_text(indices)[i] + " "

In [179]:
content

'What is AI? Artificial Intelligence helps in the analysis and decoding of neural activity data. AI, in conjunction with Brain-Computer Interfaces, receives ongoing updates of internal parameters, including data on pulse durations, amplitudes, stimulation/recording densities, and electrical properties of neural tissues. Once equipped with this information, AI algorithms swiftly discern valuable insights and logical patterns within the data, enabling them to generate the desired functional results in real time. One of the applications that has resulted from this is the ability to control a computer cursor. This can help people with disabilities to operate a personal computer to get various tasks done. Another crucial application has been in neuroprosthetics and limb rehabilitation. What does this imply in the context of healthcare, equality, and our humanity itself? All this has been possible mainly due to the advancements in Artificial Intelligence technology that have propelled the de

In [180]:
from openai import OpenAI
client = OpenAI()



response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": content}
  ]
)
print(response.choices[0].message.content)

{
  "AI": {
    "Definition": "Artificial intelligence (AI) is a technology that enables machines to learn from experience, adapt to new inputs, and perform human-like tasks.",
    "Applications": [
      "Analysis and decoding of neural activity data",
      "Brain-Computer Interfaces updates",
      "Control of computer cursor for people with disabilities",
      "Neuroprosthetics and limb rehabilitation"
    ],
    "Implications": {
      "Healthcare": "AI in conjunction with Brain-Computer Interfaces has significant implications for healthcare, enabling improved quality of life for paralyzed patients and assisting people with disabilities in their daily activities.",
      "Equality": "By providing assistive technologies like AI-powered Brain-Computer Interfaces, it helps in promoting equality by giving individuals with disabilities the tools to independently perform tasks that they might otherwise find challenging.",
      "Humanity": "The development of AI technology in the conte